In [ ]:
from evaluate import load

import math
import torch
import numpy as np
from torch.utils.data import DataLoader
from whisper.normalizers.basic import BasicTextNormalizer
from datasets import load_dataset, DatasetDict
from huggingface_hub import login
from transformers import AutoTokenizer
from transformers import GenerationConfig
from transformers import BitsAndBytesConfig
from transformers import Seq2SeqTrainer
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments 
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from peft import IA3Config
from evaluate import load


login(token="")

bleu_metric = load("sacrebleu")
comet_metric = load("comet")

DEBUG_MODE = False
DEBUG_FRACTION = 0.05

/home/alumno.upv.es/gdipal1/envs/ta-project/lib/python3.10/site-packages/torchmetrics/utilities/imports.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
/home/alumno.upv.es/gdipal1/envs/ta-project/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:197: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [2]:
raw_datasets = load_dataset("csv", data_files = "ASR_csvs/ASR_large.csv")
def is_complete(example):
    for v in example.values():

        if v is None:
            return False

        if isinstance(v, float) and math.isnan(v):
            return False

        if isinstance(v, str) and v.strip() == "":
            return False

    return True
raw_datasets = raw_datasets.filter(is_complete)

if DEBUG_MODE:
    raw_datasets = DatasetDict({
        split: raw_datasets[split]
        .shuffle(seed=42)
        .select(range(int(len(raw_datasets[split]) * DEBUG_FRACTION)))
        for split in raw_datasets.keys()
    })

print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'hypothesis_simple', 'hypothesis_traditional', 'translation', 'clean_sentence', 'clean_hypothesis_simple', 'clean_hypothesis_traditional', 'clean_translation', 'training_time'],
        num_rows: 242
    })
})


In [3]:
max_tok_length = 275
checkpoint_nllb = "facebook/nllb-200-distilled-600M"
checkpoint_mbart = "facebook/mbart-large-50-many-to-many-mmt"

src_code = "zho_Hans"
tgt_code = "eng_Latn"

src_code_mbart_trad = "zh_CN"
src_code_mbart_simple = "zh_CN"
tgt_code_mbart = "en_XX"

tokenizer_nllb = AutoTokenizer.from_pretrained(
    checkpoint_nllb, 
    padding=True, 
    pad_to_multiple_of=8, 
    src_lang=src_code, 
    tgt_lang=tgt_code, 
    truncation=False, 
    max_length=max_tok_length,
)

tokenizer_mbart_simple = AutoTokenizer.from_pretrained(
    checkpoint_mbart,
    padding=True,
    pad_to_multiple_of=8,
    src_lang=src_code_mbart_simple,
    tgt_lang=tgt_code_mbart,
    truncation=True,
    max_length=max_tok_length,
)

tokenizer_mbart_trad = AutoTokenizer.from_pretrained(
    checkpoint_mbart,
    padding=True,
    pad_to_multiple_of=8,
    src_lang=src_code_mbart_trad,
    tgt_lang=tgt_code_mbart,
    truncation=True,
    max_length=max_tok_length,
)

normalizer = BasicTextNormalizer()


In [4]:
def preprocess_function(sample, tokenizer, src, tgt):
    model_inputs = tokenizer(
        sample[src],
        text_target=sample[tgt],
        padding="max_length",
        truncation=True,
        max_length=max_tok_length
    )
    return model_inputs


In [5]:
dataset_nllb_simple = raw_datasets.map(
    lambda x: preprocess_function(x, tokenizer_nllb, src="hypothesis_simple", tgt="translation"),
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)
dataset_nllb_trad = raw_datasets.map(
    lambda x: preprocess_function(x, tokenizer_nllb, src="hypothesis_traditional", tgt="translation"),
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)
dataset_mbart_simple = raw_datasets.map(
    lambda x: preprocess_function(x, tokenizer_mbart_simple, src="hypothesis_simple", tgt="translation"),
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)
dataset_mbart_trad = raw_datasets.map(
    lambda x: preprocess_function(x, tokenizer_mbart_trad, src="hypothesis_traditional", tgt="translation"),
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

In [7]:
dataset_nllb_simple_clean = raw_datasets.map(
    lambda x: preprocess_function(x, tokenizer_nllb, src="clean_hypothesis_simple", tgt="clean_translation"),
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)
dataset_nllb_trad_clean = raw_datasets.map(
    lambda x: preprocess_function(x, tokenizer_nllb, src="clean_hypothesis_traditional", tgt="clean_translation"),
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)
dataset_mbart_simple_clean = raw_datasets.map(
    lambda x: preprocess_function(x, tokenizer_mbart_simple, src="clean_hypothesis_simple", tgt="clean_translation"),
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)
dataset_mbart_trad_clean = raw_datasets.map(
    lambda x: preprocess_function(x, tokenizer_mbart_trad, src="clean_hypothesis_traditional", tgt="clean_translation"),
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

In [50]:
def build_model(
    checkpoint,
    quantization_config,
):
    model = AutoModelForSeq2SeqLM.from_pretrained(
        checkpoint,
        quantization_config=quantization_config,
        device_map="auto",
    )
    
    generation_config = GenerationConfig.from_pretrained(
        checkpoint,
    )
    
    return model, generation_config

In [51]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [52]:
model_nllb, generation_config_nllb = build_model(
    checkpoint_nllb,
    quantization_config,
)

model_mbart, generation_config_mbart = build_model(
    checkpoint_mbart,
    quantization_config,
)

In [ ]:
test_batch_size = 32

def evaluate_model(
    model, 
    generation_config, 
    dataset, 
    tokenizer,
    num_beams
):
    model.eval()
    
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        padding='longest',
        return_tensors="pt"
    )
    
    dataloader = DataLoader(
        dataset["train"],
        batch_size=test_batch_size,
        collate_fn=data_collator,
    )
    
    input_sequences = []
    pred_sequences = []
    label_sequences = []
    
    for batch in dataloader:
        with torch.no_grad():
            output_batch = model.generate(
                generation_config=generation_config, 
                input_ids=batch["input_ids"].cuda(), 
                attention_mask=batch["attention_mask"].cuda(), 
                forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_code), 
                max_length=max_tok_length, 
                num_beams=num_beams, 
                do_sample=False,
            )
            
            input_sequences.extend(batch["input_ids"].cpu().numpy())
            pred_sequences.extend(output_batch.cpu().numpy())
            label_sequences.extend(batch["labels"].cpu().numpy())
    
    dedoced_inputs = tokenizer.batch_decode(input_sequences, skip_special_tokens=True)
    decoded_preds = tokenizer.batch_decode(pred_sequences, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(label_sequences, skip_special_tokens=True)
    
    return dedoced_inputs, decoded_preds, decoded_labels


In [ ]:
def evaluate_metrics(sample):
    inputs, preds, labels = sample
    inputs = [normalizer(text) for text in inputs]
    preds = [normalizer(text) for text in preds]
    labels = [normalizer(text) for text in labels]
    results = {
        "bleu": bleu_metric.compute(predictions=preds, references=[labels[i] for i in range(len(labels))])["score"],
        "comet": comet_metric.compute(predictions=preds, references=labels, sources=inputs)["mean_score"] * 100,
    }
    return results

In [ ]:
import pandas as pd

# Creiamo lista di configurazioni con più beams
cfg_list = []

for model_name, base_cfg in {
    "nllb_simple": (model_nllb, generation_config_nllb, dataset_nllb_simple, tokenizer_nllb),
    "nllb_trad": (model_nllb, generation_config_nllb, dataset_nllb_trad, tokenizer_nllb),
    "mbart_simple": (model_mbart, generation_config_mbart, dataset_mbart_simple, tokenizer_mbart_simple),
    "mbart_trad": (model_mbart, generation_config_mbart, dataset_mbart_trad, tokenizer_mbart_trad),
    "nllb_simple_clean": (model_nllb, generation_config_nllb, dataset_nllb_simple_clean, tokenizer_nllb),
    "nllb_trad_clean": (model_nllb, generation_config_nllb, dataset_nllb_trad_clean, tokenizer_nllb),
    "mbart_simple_clean": (model_mbart, generation_config_mbart, dataset_mbart_simple_clean, tokenizer_mbart_simple),
    "mbart_trad_clean": (model_mbart, generation_config_mbart, dataset_mbart_trad_clean, tokenizer_mbart_trad),
}.items():
    model, gen_cfg, dataset, tokenizer = base_cfg
    for num_beams in [1, 4]:
        cfg_list.append({
            "model_name": model_name,
            "model": model,
            "generation_config": gen_cfg,
            "dataset": dataset,
            "tokenizer": tokenizer,
            "num_beams": num_beams
        })

results_list = []

for cfg_item in cfg_list:
    name = cfg_item["model_name"]
    num_beams = cfg_item["num_beams"]
    print(f"Evaluating model: {name} with num_beams={num_beams}")

    inputs, preds, labels = evaluate_model(
        model=cfg_item["model"],
        generation_config=cfg_item["generation_config"],
        dataset=cfg_item["dataset"],
        tokenizer=cfg_item["tokenizer"],
        num_beams=num_beams,
    )
    
    metrics = evaluate_metrics((inputs, preds, labels))
    
    results_list.append({
        "model_name": name,
        "num_beams": num_beams,
        "bleu": metrics["bleu"],
        "comet": metrics["comet"]
    })
    print(metrics)

df_results = pd.DataFrame(results_list)
df_results.to_csv("cascade_baseline.csv", index=False)
print("Results saved to cascade_baseline.csv")

Evaluating model: M2M100ForConditionalGeneration with tokenizer: NllbTokenizerFast


KeyboardInterrupt: 